 ## Geometric mean
 _CYBR 304 & MATH 420_ <br>
 Spring 2024 <br>

The _geometric mean_ $\mathrm{GM}$ of nonnegative numbers $x_1, x_2, \dots, x_n$ is the nth root of their product. Specifically
$$
   \mathrm{GM}(x_1, x_2, \dots x_n) = \left( \prod_{k=1}^n x_k \right)^{1/n}.
$$
The geometric mean is often used in finance to find investment returns and it has other applications in engineering as well. In this worksheet, we'll write Julia code that evaluates the geometric mean, paying particular attention to overflow, underflow, and accuracy.

One property of the geometric mean is that $\mathrm{GM}(x,x) = x$ is an identity for all nonnegative numbers $x$. We'll use this identity as a test for our Julia code. Another property of the geometric mean is that any one of the numbers $x_1, x_2, \dots, x_n$ is zero, the geometric mean of these numbers is zero.

Our first effort for a Julia function for the geometric mean is little more than a direct translation of the definition. Here we use the Julia function `prod` to evaluate the product of the members of an array.

When the input array is empty, we need to decide what to do. For an empty array, the Julia function `prod` returns the multiplicative identity for the type of the array; for example

In [2]:
(prod(Int64[]), prod(Float64[]))

(1, 1.0)

Further, in Julia, `1/0` evaluates to `Inf`, and the interderminate form `1^Inf` evaluates to 1. If we accept these Julia conventions, we can write our code as

In [4]:
"""
    geometric_mean(a::Array)

Compute the geometric mean of the elements in the input array `a`. 
"""
function geometric_mean(a::Array{T}) where T <: Any
   prod(a)^(1/length(a)) 
end;

Three simple tests show that our function works OK.

In [6]:
(geometric_mean([5,20]), sqrt(5*20))

(10.0, 10.0)

In [7]:
(geometric_mean([1,2,3,4]), (1*2*3*4)^(1/4))

(2.2133638394006434, 2.2133638394006434)

In [8]:
(geometric_mean([0,1,2,3,4]), (0*1*2*3*4)^(1/5))

(0.0, 0.0)

In [9]:
geometric_mean([Inf,4,5])

Inf

In [10]:
geometric_mean([Inf, 0])

NaN

For empty arrays, we have

In [12]:
geometric_mean(Int64[])

1.0

In [13]:
geometric_mean(Float64[])

1.0

In [14]:
geometric_mean([2^(-538), 2^(-538)])

0.0

In [15]:
geometric_mean([-1,-2,-4])

LoadError: DomainError with -8.0:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.

In [16]:
geometric_mean([2+im, 6.7 + im])

3.7113075237539723 + 1.172093654906827im

In [17]:
geometric_mean([1.0e155, 1.0e155])

Inf

This result violates the identity $GM(x,x) = x$.  We really should do better.  A typical way to fix this overflow problem is to use the fact that the logarithm of a product of positive numbers is the sum of the logarithms.  This gives an alternative formula for the geometric sum
$$
   GM(x_1, x_2, \dots, x_n) = \exp(\frac{1}{n}  \left( \sum_{k=1}^n \ln(x_k) \right))
$$
Alternatively, we can use the base-two logarithm and base two exponentation. For floating point numbers in binary form, this is possilby the most natural choice.

A simple implementation of this method is

The Julia sum function returns zero for an empty sum; for example

In [20]:
sum(Float64[])

0.0

But dividing zero by the length of an empty array yields `NaN`.  Arguably for this code, we should special case an empty array input

In [22]:
"""
    geometric_mean_log(a)

Compute the geometric mean of the elements in the input array `a` using a logarithmic transformation to avoid overflow. When the 
array `a` is empty or when the array contains a negative entry, throw an ArgumentError.
"""
function geometric_mean_log(a::Array{T}) where T <: Any
    n = length(a)
    if n==0
        one(T)
    else
      2^(sum(map(log2,a))/n)
    end
end;

In [23]:
(geometric_mean_log([5,20]), sqrt(5*20))

(10.000000000000002, 10.0)

In [24]:
(geometric_mean_log([1,2,3,4]), (1*2*3*4)^(1/4))

(2.213363839400643, 2.2133638394006434)

We have resolved the overflow problem, but arguably our function isn't as accurate as it might be; for example

In [26]:
geometric_mean_log([1.0e155, 1.0e155])

1.000000000000028e155

Because $\log(0)$ is undefined, you might think that `geometric_mean_log` misbehaves when one or more argument is zero, but it doesn't.

In [28]:
geometric_mean_log([0,1,2,3])

0.0

To see what happens, we can work through the calculation one step at a time

In [30]:
x = map(log,[0,1,2,3])

4-element Vector{Float64}:
 -Inf
   0.0
   0.6931471805599453
   1.0986122886681098

In [31]:
x = sum(x)

-Inf

In [32]:
x = x/4

-Inf

In [33]:
exp(x)

0.0

In Julia log(0) = -Inf and exp(-Inf) = 0.


Also, the code has resolved the underflow problem; for example

In [36]:
geometric_mean_log([2^(-538), 2^(-538), 2^(-538)])

1.1113793747425387e-162

In [37]:
sum(map(log2, Float64[]))

0.0

The code does seem to return one for an empty array

In [39]:
geometric_mean_log(Float64[])

1.0

In [40]:
"""
    geometric_mean(a)

Compute the geometric mean of the elements in the input array `a`. 
"""
function geometric_mean2(a::Array{Float64})
    n = length(a)
    if n==0 
        one(Float64)
    else 
      e = 0
      s = one(eltype(a))
      for x in a
        s *= x
        e += exponent(s)
        s = significand(s)
      end
     exp2(e/n)*s^(1/n)
    end
end
    

geometric_mean2

In [41]:
geometric_mean2(Float64[])

1.0

In [42]:
geometric_mean2([1.0e155, 1.0e155])

1.0e155

In [43]:
geometric_mean2([1.0e308, 1.0e308, 1.0e308])

1.0e308

There is a standard Julia package that has code for the geometric mean.  To use it, we need to use the package manager to download and install it. Once we have done that one time, to use the package, we only need to load it. Additionally, to compare running times, we'll use the `BenchmarkTools` package.

In [45]:
using StatsBase, BenchmarkTools

Specifically, the function name is `geomean`.  It handle the underflow and overflow tests OK, but arguably, and overflow test shows that `geomean` isn't as accurate as it might be

In [47]:
geomean([2^(-538), 2^(-538)])

1.1113793747425612e-162

This result should be zero, but it is not

In [49]:
(geomean([exp2(155) for k=1:10^7]) , exp2(155))

(4.5671926166570814e46, 4.567192616659072e46)

Our code returns an accurate value

In [51]:
(geometric_mean2([exp2(155) for k=1:10^7]) , exp2(155))

(4.567192616659072e46, 4.567192616659072e46)

In [52]:
geometric_mean2([2,20,Inf])

LoadError: DomainError with Inf:
Cannot be NaN or Inf.

In [53]:
L = rand(Float64,10^7);

In [54]:
@btime x = geometric_mean2(L)

  35.652 ms (1 allocation: 16 bytes)


0.3679514396038032

In [55]:
@btime y = geomean(L)

  89.300 ms (1 allocation: 16 bytes)


0.36795143960380317

For the fastest speed, we should stick with our first and overflow and underflow defective code

In [57]:
@btime x = geometric_mean(L)

  4.347 ms (1 allocation: 16 bytes)


0.0